In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

print(".... \nBuilding auth code url\n.... \n")

'''
Register app on strava.com/setings/api to get the *C_ID, *C_SECRET
'''
required_vars = ["STRAVA_C_ID", "STRAVA_C_SECRET", "STRAVA_REDIRECT"]

# Validate .env vars
missing_vars = [var for var in required_vars if os.getenv(var) is None]

if missing_vars:
    raise ValueError(f"Missing environment variables: {', '.join(missing_vars)}")
    
'''
Build the url to get permissions, and get auth code
'''
def build_url():
    client_id = os.getenv("STRAVA_C_ID")
    client_secret = os.getenv("STRAVA_C_SECRET")
    redirect_url = os.getenv("STRAVA_REDIRECT")
    
    root_url = "https://www.strava.com/oauth/authorize"
    scope = "activity:read_all"
    response_type = "code"
    print(f"Click here -> {root_url}?client_id={client_id}&response_type={response_type}&scope={scope}&redirect_uri={redirect_url}")
if __name__ == "__main__":
    print(f"READ ME:\n1. Follow the auth code url\n2. Accept Access Request\n3. Look to the url and copy `code=` into .env.STRAVA_C_AUTH_CODE\n4. Move on to next step\n")
    build_url()


In [ ]:
import logging
import requests
import json
from dotenv import load_dotenv
import os

logger = logging.getLogger(__name__)
logging.basicConfig(filename='strava_data.log', level=logging.INFO)

load_dotenv()

required_vars = ["STRAVA_C_ID", "STRAVA_C_SECRET", 'STRAVA_C_AUTH_CODE']

# Validate .env vars
missing_vars = [var for var in required_vars if os.getenv(var) is None]

if missing_vars:
    raise ValueError(f"Missing environment variables: {', '.join(missing_vars)}")
    
def getAccessToken ():
    cl_id = os.getenv("STRAVA_C_ID")
    cl_secret = os.getenv("STRAVA_C_SECRET")
    cl_auth_code = os.getenv("STRAVA_C_AUTH_CODE")
    
    token_url = "https://www.strava.com/oauth/token"
    token_params = {
        "client_id": cl_id,
        "client_secret": cl_secret,
        "code": cl_auth_code,
        "grant_type": "authorization_code"
    }

    response = requests.post(token_url, data=token_params)
    token_data = response.json()

    if 'access_token' not in token_data:
        print("Error obtaining access token:")
        print(token_data)
        exit()
    return token_data['access_token']


if __name__ == "__main__":
    access_token = getAccessToken();
    print("Save me in .env file under STRAVA_C_ACCESS_CODE\n")
    print(f"Access token: {access_token}\n")
    

In [ ]:
import logging
import requests
import json
from dotenv import load_dotenv
import os

logger = logging.getLogger(__name__)
logging.basicConfig(filename='strava_data.log', level=logging.INFO)

def main():
    cl_access_code = os.getenv("STRAVA_C_ACCESS_CODE")
    if (cl_access_code is None):
        raise ValueError(f"Missing environment variables: STRAVA_C_ACCESS_CODE")

    activities_url = "https://www.strava.com/api/v3/athlete/activities"
    headers = {"Authorization": f"Bearer {cl_access_code}"}
    params = {"per_page": 10, "page": 1}  # Get first 10 activities

    response = requests.get(activities_url, headers=headers, params=params)

    activities_path = f"./strava/activities.json"
    
    if response.status_code == 200:
        activities = response.json()

        with open(activities_path, 'w') as f:
            json.dump(activities, f)
        print(f"Wrote activties to: {activities_path} \n")
        
        print(f"Received {len(activities)} activities: \n")
        for activity in activities:
            print(f"Activity: {activity.get('name', 'Unnamed Activity')}")
            print(f"Type: {activity.get('type', 'N/A')}")
            print(f"Distance: {activity.get('distance', 0):.2f} meters")
            print(f"Date: {activity.get('start_date_local', 'N/A')}\n")
    else:
        print(f"Error retrieving activities: {response.status_code}")
        print(response)
if __name__ == "__main__":
    main()
    